In [1]:
import tensorflow as tf
import tensorflow_hub
import cv2

import os
import numpy as np

import matplotlib.pyplot as plt

In [2]:
trn_name = 'dataset/asl_alphabet_train'
trn_dirs = [os.path.join(trn_name, sign) for sign in os.listdir(trn_name)]
trn_fnames = [os.listdir(sign_img) for sign_img in trn_dirs]

tst_name = 'dataset/asl_alphabet_test'
tst_dirs = [os.path.join(tst_name, sign) for sign in os.listdir(tst_name)]
tst_fnames = [os.listdir(sign_img) for sign_img in tst_dirs]

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


BATCH_SIZE = 100
IMAGE_SHAPE = (224, 224)


image_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2,
    rescale=1. / 255)


train_generator = image_generator.flow_from_directory(
    trn_name,
    subset='training',
    batch_size=BATCH_SIZE,
    target_size=IMAGE_SHAPE,
    class_mode="categorical")

vldn_generator = image_generator.flow_from_directory(
    trn_name,
    subset='validation',
    batch_size=BATCH_SIZE,
    target_size=IMAGE_SHAPE,
    class_mode="categorical")

test_datagen = ImageDataGenerator(rescale=1. / 255)
tst_generator = image_generator.flow_from_directory(
    tst_name,
    batch_size=BATCH_SIZE,
    target_size=IMAGE_SHAPE,
    class_mode="categorical")


Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.
Found 29 images belonging to 29 classes.


In [4]:
train_size = train_generator.samples
vldn_size = vldn_generator.samples

In [17]:
model_url = "https://tfhub.dev/tensorflow/efficientnet/lite4/feature-vector/2"

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=IMAGE_SHAPE + (3,)),
    tensorflow_hub.KerasLayer(model_url, trainable=False),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(290, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(29, activation='softmax',
                          kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])

model.build((None,) + IMAGE_SHAPE + (3,))
model.summary()

model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.005, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=['acc'])


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 1280)              11837936  
                                                                 
 dropout_4 (Dropout)         (None, 1280)              0         
                                                                 
 dense_4 (Dense)             (None, 290)               371490    
                                                                 
 dropout_5 (Dropout)         (None, 290)               0         
                                                                 
 dense_5 (Dense)             (None, 29)                8439      
                                                                 
Total params: 12,217,865
Trainable params: 379,929
Non-trainable params: 11,837,936
_________________________________________________________________


In [18]:
checkpoint_path = "./checkpoints/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

print(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=20)


./checkpoints/cp.ckpt


In [21]:
# training
if os.path.exists('./checkpoints') == True:
  latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
  model.load_weights(checkpoint_path)

history = model.fit(
    train_generator,
    validation_data=vldn_generator,
    epochs=1,
    validation_steps=10,
    verbose=1,
    callbacks=[cp_callback])

model.save('./model_saves')


 19/696 [..............................] - ETA: 13:30 - loss: 3.4224 - acc: 0.0679
Epoch 00001: saving model to ./checkpoints\cp.ckpt
 39/696 [>.............................] - ETA: 13:21 - loss: 3.1173 - acc: 0.1508
Epoch 00001: saving model to ./checkpoints\cp.ckpt
 59/696 [=>............................] - ETA: 13:00 - loss: 2.8757 - acc: 0.2247
Epoch 00001: saving model to ./checkpoints\cp.ckpt
 79/696 [==>...........................] - ETA: 12:37 - loss: 2.6447 - acc: 0.2859
Epoch 00001: saving model to ./checkpoints\cp.ckpt
 99/696 [===>..........................] - ETA: 12:14 - loss: 2.4484 - acc: 0.3380
Epoch 00001: saving model to ./checkpoints\cp.ckpt
119/696 [====>.........................] - ETA: 11:50 - loss: 2.2767 - acc: 0.3852
Epoch 00001: saving model to ./checkpoints\cp.ckpt
139/696 [====>.........................] - ETA: 11:25 - loss: 2.1330 - acc: 0.4217
Epoch 00001: saving model to ./checkpoints\cp.ckpt
159/696 [=====>........................] - ETA: 11:01 - loss: 

INFO:tensorflow:Assets written to: ./model_saves\assets


In [24]:
# testing
from tensorflow.keras.preprocessing import image

test_imgs = [
    'dataset/asl_alphabet_train/X/X2.jpg'
]

test_imgs = [os.path.join(file, os.listdir(file)[0]) for file in tst_dirs]

imgs = [np.expand_dims(
    image.img_to_array(
        image.load_img(img, target_size=IMAGE_SHAPE)
    ), axis=0)
    for img in test_imgs]

# pass the list of multiple images np.vstack()
images = np.vstack(imgs)
classes = model.predict(images, batch_size=10)
# print the classes, the images belong to
all_classes = os.listdir(trn_name)
final_results = []
for probablities in classes:
    final_results.append(all_classes[np.argmax(probablities)]) 
print(final_results)

['B',
 'S',
 'C',
 'D',
 'X',
 'N',
 'E',
 'nothing',
 'G',
 'nothing',
 'W',
 'space',
 'X',
 'R',
 'G',
 'O',
 'N',
 'nothing',
 'O',
 'D',
 'Q',
 'X',
 'R',
 'K',
 'K',
 'space',
 'W',
 'V',
 'nothing']

In [40]:
# Live Detection
def predict_asl(frame) -> str:
	frame = image.smart_resize(frame, (224, 224))
	img = np.expand_dims(image.img_to_array(frame), axis=0)
	predicted_asl = model.predict(img)
	return all_classes[np.argmax(predicted_asl)]

alphabet = ''
cap = cv2.VideoCapture(0)
frame_number = 0
while(True):
	ret, frame = cap.read()
	cv2.putText(frame,
             alphabet,
             (50, 50),
             font, 1,
             (0, 255, 255),
             2,
             cv2.LINE_4)
	cv2.imshow("Webcam", frame)
	while (frame_number == 180):
		alphabet = predict_asl(frame)
		print(alphabet)
		font = cv2.FONT_HERSHEY_SIMPLEX
		frame_number = 0
	frame_number += 1


	ch = cv2.waitKey(1)
	if ch & 0xFF == ord('q'):
		cap.release()
		break

cv2.destroyAllWindows()

E
